In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import google.generativeai as genai  # Importing the Google Generative AI module from the google package
import os

c:\Users\Vibhor\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Setting the API key for Google Generative AI service by assigning it to the environment variable 'GOOGLE_API_KEY'
api_key = os.environ['GOOGLE_API_KEY'] = "xx-xxxxx"

# Configuring Google Generative AI module with the provided API key
genai.configure(api_key=api_key)

In [3]:
class Model:
    # Class method to instantiate a model
    @classmethod
    def model(cls,model_name=None,temp=None):
        try:
            # Attempt to initialize and return a ChatGoogleGenerativeAI model
            return ChatGoogleGenerativeAI(model=model_name,temperature=temp, google_api_key=api_key)
        except Exception as e:
            # Return any exception that occurs during instantiation
            return e

class Chain:
    # Class method to create a chain
    @classmethod
    def chain(cls, model_name=None, temp=None,prompt=None):
        try:
            # Attempt to create an LLMChain instance using the specified model and prompt
            return LLMChain(llm=Model.model(model_name,temp),prompt=prompt)
        except Exception as e:
            # Return any exception that occurs during chain creation
            return e

In [4]:

class PromptTemplates:
    """
    This class provides reusable prompt templates for different prompting strategies:
    1. Keyword Extraction (RAG style factual QA)
    2. Chain-of-Thought reasoning (step-by-step logical answering)
    3. Verification prompts (double-check factual correctness)
    """

    @classmethod
    def key_word_extraction(cls):
        """
        Use Case:
        ---------
        Ideal for **RAG (Retrieval-Augmented Generation)** tasks.
        It ensures that the model answers strictly based on retrieved document context
        without adding external knowledge or assumptions.

        Example Usage:
        - Document Question Answering.
        - Information extraction tasks where accuracy is critical.
        - Preventing hallucinations in factual QA.

        Returns:
            A PromptTemplate instructing the model to:
            - ONLY use facts explicitly from context.
            - Avoid external knowledge/assumptions.
            - Reply explicitly if the answer is not present.
        """
        try:
            prompt = """
                    You are an intelligent assistant.

                    Below is the information retrieved from the document:

                    {context}

                    Now, answer the following question strictly based on the above information:

                    Question: {question}

                    Guidelines:
                    - ONLY use facts explicitly mentioned in the context.
                    - Do NOT use external knowledge.
                    - Do NOT make assumptions.
                    - If the answer is not present, reply: "The document does not contain this information."

                    Provide your answer:
                    """
            return PromptTemplate(template=prompt.strip(), input_variables=["context", "question"])
        except Exception as e:
            return e

    @classmethod
    def chain_of_thoughts(cls):
        """
        Use Case:
        ---------
        Implements **Chain-of-Thought (CoT) prompting**.
        Helps model break down complex questions and reason step by step logically based ONLY on the document content.

        Example Usage:
        - Complex document QA needing logical reasoning.
        - Scenarios where multi-step analysis improves answer accuracy.
        - Summarization tasks requiring explanation or derivation.

        Returns:
            A PromptTemplate guiding the model to:
            - Think step by step.
            - Extract, analyze, and logically derive the answer.
            - Avoid external knowledge and assumptions.
            - Avoid bullet points; present a smooth, clear final answer.
        """
        try:
            prompt = """
                       You are a thoughtful assistant.

                        Here is the document content:

                        {context}

                        Question: {question}

                        Think step by step based ONLY on the provided content.
                        Extract the relevant information, analyze it logically, and derive a clear answer.

                        Rules:
                        - Do NOT use any outside knowledge.
                        - Do NOT assume facts not explicitly stated.
                        - Just only display and elaborate the final answer.

                        Begin reasoning:
                    """
            return PromptTemplate(template=prompt.strip(), input_variables=["context", "question"])
        except Exception as e:
            return e

    @classmethod
    def verification_prompt(cls):
        """
        Use Case:
        ---------
        Implements **Verification prompting technique**.
        Helps double-check the factual correctness of model-generated answers by verifying if they are supported by the document content.

        Example Usage:
        - Post-processing stage after answer generation to ensure factual alignment.
        - Use in sensitive domains like healthcare, legal, finance.
        - To flag unverifiable or hallucinated answers.

        Returns:
            A PromptTemplate that:
            - Cross-verifies if the provided answer is directly supported by document context.
            - Gives a binary "Verified" or "Cannot verify" response.
        """
        try:
            prompt = """
                        You are a careful assistant.

                        Here is the document content:

                        {context}

                        Question: {question}

                        Provide the answer **based ONLY on the above document content**.

                        Verify if the answer can be **directly supported by the content**.
                        - If YES, state: "Verified: Answer supported by the document."
                        - If NO, state: "Cannot verify: The document does not contain enough information."

                        Answer:
                    """
            return PromptTemplate(template=prompt.strip(), input_variables=["context", "question"])
        except Exception as e:
            return e


In [5]:
class ConversationAgent:
    # Constructor to initialize the ConversationAgent with model name and temperature
    def __init__(self, model_name=None, temp=None):
        self.model = model_name
        self.temperature = temp
    
    # Method to declare chains based on a given prompt template
    def declare_chains(self, prompt_template):
        try:
            # Instantiate a chain using the specified model and prompt template
            return Chain.chain(model_name=self.model, temp=self.temperature, prompt=prompt_template)
        except Exception as e:
            # Return any exception that occurs during chain declaration
            return e
    
    # Method to run the conversation agent with user input and prompt
    def run(self, user_text=None, prompt=None):
        # Invoke the declared chain with user input and return the response
        response = self.declare_chains(prompt).invoke(user_text)
        return response['text']

In [6]:
# Create ConversationAgent
agent = ConversationAgent(model_name="gemini-1.5-flash", temp=0)

In [7]:
# Sample context and question
context = """The Eiffel Tower is one of the most famous landmarks in Paris, France. It was designed by Gustave Eiffel and completed in 1889."""
question = "Who designed the Eiffel Tower?"

# Define your RAG Prompt Template
rag_prompt = PromptTemplates.key_word_extraction()
# Prepare input variables
user_input = {
    "context": context,
    "question": question
}

# Run
response = agent.run(user_text=user_input, prompt=rag_prompt)
print(response)

C:\Users\Vibhor\AppData\Local\Temp\ipykernel_14824\3967111467.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=Model.model(model_name,temp),prompt=prompt)


Gustave Eiffel



In [8]:
# Example document content and question
document_content = """
John is planning to attend a conference in New York. He booked his flight for March 25th and reserved a hotel for 3 nights.
"""
document_question = "On what date will John check out from the hotel?"
user_input = {
    "context": document_content,
    "question": document_question
}
# Define COT Prompt Template
cot_prompt = PromptTemplates.chain_of_thoughts()

# Run
response = agent.run(user_text=user_input, prompt=cot_prompt)
print(response)

John will check out of the hotel on March 28th.  This is because he booked a hotel for 3 nights starting on March 25th (the day he arrives).  Three nights after March 25th is March 28th.

